In [5]:
import copy
from pathlib import Path
import os
import numpy as np
from numpy.lib.format import open_memmap
import sys
import yaml

import spikeextractors as se

#from  RecordingExtractor
#from spikesorters import BaseSorter
from ..basesorter import BaseSorter
from ..sorter_tools import recover_recording

from ..utils.shellscript import ShellScript


from shellscript import ShellScript

try:
    import yass
    HAVE_YASS = True
except ImportError:
    HAVE_YASS = False

print ("HAVE_YASS: ", HAVE_YASS)
        

ImportError: attempted relative import with no known parent package

In [2]:
# yasssorter class
class YassSorter(BaseSorter):
    """ 
    """

    sorter_name = 'yass'
    requires_locations = False
    verbose = False
    is_installed = HAVE_YASS

    # _default_params comes from default Yass config.yaml file; 
    #  Should be autoloaded from yass_installation_directory/samples/10chan/config.yaml
    #  - for now a copy is saved here: spikesorters/spikesorters/yass/config.yaml
    
#     _default_params = {
#         'adjacency_radius': 100,  # Channel neighborhood adjacency radius corresponding to geom file
#         'filter': True,
#         }

    # ALESSIO: Move thsi file locally to the yass directory;
    default_config_fname = './config_sample.yaml'
    with open(default_config_fname) as f:
        _default_params = yaml.load(f, Loader=yaml.FullLoader)   
    
    _params_description = {
        'data': " ...",
    }

    # 
    sorter_description = """Yass uses Neural Networks and SuperResolution Deconvolution to sort Retinal and 
                        cortical data. 
                        For more information see https://www.biorxiv.org/content/10.1101/2020.03.18.997924v1
                        """

    installation_mesg = """\nTo Install and Use Yass follow the wiki: 
                            https://github.com/paninski-lab/yass/wiki
                        """

    def __init__(self, **kargs):
        print (**kargs)
        #BaseSorter.__init__(self, rec, output_folder)
        self.params = self._default_params
        
    @classmethod
    def is_installed(cls):
        #return HAVE_YASS
        #return check_if_installed(cls.kilosort2_path)
        return self.HAVE_YASS
    
    @staticmethod
    def get_sorter_version():
        return yass.__version__

    # NEED TO CHANGE THIS FOR YASS ALSO
    # n_chan = recording.get_num_channels()
    # n_frames = recording.get_num_frames()
    # chunk_size = 2 ** 24 // n_chan
    
    # also make a default config file for
    # https://github.com/SpikeInterface/spikesorters/blob/master/spikesorters/spyking_circus/config_default.params
    # {} reserved for params that need to be updated at run time;
    
    # this function parses params and creates config file and binary data and geometry file also
    # and saving everything to the output folder
    # Cat: this function changes the minimum required default values; 
    def _setup_recording(self, recording, output_folder):
        p = self.params
        source_dir = Path(output_folder).parent

        #################################################################
        #################### UPDATE ROOT FOLDER #########################
        #################################################################
        # float(self._recording.sample_rate.rescale('Hz').magnitude)
        self.params['data']['root_folder'] = output_folder
        #self.params['data']['geometry'] = 'geom.csv'
        
        #################################################################
        #################### GEOMETRY FILE GENERATION ###################
        #################################################################
        # save prb file
        # note: only one group here, the split is done in basesorter
        probe_file_csv = os.path.join(output_folder,'geom.csv')
        probe_file_txt = os.path.join(output_folder,'geom.txt')
        # ALESSIO .saveto probe saved .prb file; have to d thisourselves.
        #  
        adjacency_radius = -1
        recording.save_to_probe_file(probe_file_csv, 
                                     grouping_property=None,
                                     radius=adjacency_radius)
        
        import csv

        with open(probe_file_csv) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            geom_txt = np.float32(np.vstack(csv_reader))
            np.savetxt(probe_file_txt, geom_txt)
        
        #################################################################
        #################### UPDATE SAMPLING RATE #######################
        #################################################################
        # float(self._recording.sample_rate.rescale('Hz').magnitude)
        self.params['recordings']['sampling_rate'] = recording.get_sampling_frequency()
        
        
        #################################################################
        #################### UPDATE N_CHAN  #############################
        #################################################################
        self.params['recordings']['n_channels'] = recording.get_num_channels()
        
        
        #################################################################
        #################### SAVE RAW INT16 data ########################
        #################################################################
        # ALESSIO Look at Kilosort 
        # There is alrady an extractor se.Mea1kRecordingExtractor()
        # all the functions are there already to concatenate in time;
        # multi-recording time extractor;
        
        # save binary file; THIS IS FROM KILOSORT
        input_file_path = os.path.join(output_folder, 'data.bin')
        
        recording.write_to_binary_dat_format(input_file_path, 
                                             dtype='int16', 
                                             chunk_mb=500) # time_axis=0,1 for C/F order
        
        
        #################################################################
        #################### SAVE UPDATED CONFIG FILE ###################
        #################################################################
        fname_config = os.path.join(output_folder,
                                   'config.yaml')
        with open(fname_config, 'w') as file:
            documents = yaml.dump(self.params, file)
        
        #self.fname_config = fname_config
        
        # ALESSIO:
        # Expose more config file parameters that are sensiitive:
        #  e.g. spike width; smallest cluster; min firing rates;
        #  
            
    # FUNCTION TO RUN YASS 
    #def _run(self,recording, output_folder):  # SOMETIMES want to access more information from recording in
                                               # this step
    def _run(self, recording, output_folder):
        '''
        '''
        recording = recover_recording(recording)  # allows this to run on multiple jobs (not just multi-core)
        
        if 'win' in sys.platform and sys.platform != 'darwin':
            shell_cmd = '''
                        yass sort {config}
                    '''.format(config=os.path.join(output_folder,'config.yaml'))
        else:
            shell_cmd = '''
                        #!/bin/bash
                        yass sort {config}
                    '''.format(config=os.path.join(output_folder,'config.yaml'))

        shell_script = ShellScript(shell_cmd, 
                                   script_path=os.path.join(output_folder,self.sorter_name),
                                   log_path=os.path.join(output_folder,self.sorter_name+'.log'), 
                                   verbose=self.verbose)
        shell_script.start()

        retcode = shell_script.wait()

        if retcode != 0:
            raise Exception('yass returned a non-zero exit code')
    
    # Alessio might not want to put here; 
    # better option to have a parameter "tune_nn" which 
    def train(self, recording, output_folder):
        '''
        '''
        recording = recover_recording(recording)  # allows this to run on multiple jobs (not just multi-core)
        
        if 'win' in sys.platform and sys.platform != 'darwin':
            shell_cmd = '''
                        yass train {config}
                    '''.format(config=os.path.join(output_folder,'config.yaml'))
        else:
            shell_cmd = '''
                        #!/bin/bash
                        yass train {config}
                    '''.format(config=os.path.join(output_folder,'config.yaml'))

        shell_script = ShellScript(shell_cmd, 
                                   script_path=os.path.join(output_folder,self.sorter_name),
                                   log_path=os.path.join(output_folder,self.sorter_name+'.log'), 
                                   verbose=self.verbose)
        shell_script.start()

        retcode = shell_script.wait()

        if retcode != 0:
            raise Exception('yass returned a non-zero exit code')  
            
    def NNs_update(self):
        ''' Update NNs to newly trained ones
        '''
        
        #################################################################
        #################### UPDATE CONFIG FILE TO NEW NNS ##############
        #################################################################
        self.params['neuralnetwork']['denoise']['filename']= os.path.join(
                                            output_folder, 
                                            'tmp',
                                            'nn_train',
                                            'denoise.pt')
        self.params['neuralnetwork']['detect']['filename']= os.path.join(
                                            output_folder, 
                                            'tmp',
                                            'nn_train',
                                            'detect.pt')
        
        #################################################################
        #################### SAVE UPDATED CONFIG FILE ###################
        #################################################################
        fname_config = os.path.join(output_folder,
                                   'config.yaml')
        
        with open(fname_config, 'w') as file:
            documents = yaml.dump(self.params, file)
        
        
    def NNs_default(self):
        ''' Revert to default NNs
        '''
        #################################################################
        #################### UPDATE CONFIG FILE TO NEW NNS ##############
        #################################################################
        self.params['neuralnetwork']['denoise']['filename']= 'denoise.pt'
        self.params['neuralnetwork']['detect']['filename']= 'detect.pt'
        
        #################################################################
        #################### SAVE UPDATED CONFIG FILE ###################
        #################################################################
        fname_config = os.path.join(output_folder,
                                   'config.yaml')
        
        with open(fname_config, 'w') as file:
            documents = yaml.dump(self.params, file)        
        
    
    # 
    @staticmethod
    def get_result_from_folder(output_folder):
        #sorting = se.YassSortingExtractor(folder_path=Path(output_folder) / 'tmp/output/spike_train.npy')
        sorting = se.YassSortingExtractor(folder_path=Path(output_folder))
        return sorting
    


NameError: name 'BaseSorter' is not defined

In [ ]:
# load Yass Sorter
sorter = YassSorter()
print ("yass installed: ", sorter.is_installed())
print ("yass version: ", sorter.get_sorter_version())

In [9]:
# INITIALIZE A TOY DATASET
import spikeextractors as se
rec, sort = se.example_datasets.toy_example(duration=300)
rec.get_channel_locations()

array([[1., 0.],
       [2., 0.],
       [3., 0.],
       [4., 0.]])

In [10]:
# INITIALIZE YASS SORTER OBJECT
output_folder = '/media/cat/4TBSSD/spikeinterface/'
sorter = YassSorter()


In [11]:
# RUN SETUP
sorter._setup_recording(rec, output_folder)


/home/cat/.conda/envs/sf/lib/python3.7/site-packages/ipykernel_launcher.py:96: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [6]:
# RUN TRAIN
if False:
    sorter.train(rec, output_folder)

    # UPDATE NNs to TRAINED VERSIOn
    sorter.NNs_update()


RUNNING SHELL SCRIPT: /media/cat/4TBSSD/spikeinterface/yass.sh


In [25]:
# RUN YASS
sorter._run(rec, output_folder)



RUNNING SHELL SCRIPT: /media/cat/4TBSSD/spikeinterface/yass.sh


In [27]:
# VISUALIZE RESULTS
from spikeextractors import YassSortingExtractor

d = YassSortingExtractor(output_folder)

unit_id = 3

spikes = d.get_unit_spike_train(unit_id, start_frame=20000, end_frame=400000)
print ("Unit: ", unit_id, " spikes: ", spikes)

temps = d.get_temps()
print (temps.shape)

Unit:  3  spikes:  [ 27667  44270  44394  67223  68316  80673  82989  93560 109300 151074
 179404 189923 200279 202184 203674 205694 276385 293983 304940 309969
 310093 313842 330439 332320 366638 370371 395914]
(8, 151, 4)


In [1]:
#sorter.run(rec, output_folder)




yass installed:  True
yass version:  2.0


In [4]:
# EXTRACTOR THAT LOADS DEFAULT CONFIG FILE FOUND IN DATAFILE DIRECTORY
# Default option is to send .bin binary file name and the geom.txt and config.yaml files to be hardcoded

# initatlize extractor object
extractor = YassDefaultRecordingExtractor('/home/cat/code/alessio/10chan/data.bin')
extractor.get_channel_ids()
extractor.get_geometry()
extractor.get_num_frames()
extractor.get_sampling_frequency()


20000.0

In [5]:
output_folder = '/home/cat/code/alessio/10chan/'
sorter._run(output_folder)


RUNNING SHELL SCRIPT: /home/cat/code/alessio/10chan/yass.sh


In [6]:
#se.YassSortingExtractor()

In [13]:
temp = se.YassSortingExtractor.get_unit_ids()


TypeError: get_unit_ids() missing 1 required positional argument: 'self'

In [18]:
print (temp.spike_train)

None


In [52]:
print (extractor.config)

/home/cat/code/alessio/10chan/config.yaml


In [ ]:
# toy exapmle 
rec, sort = se.example_datasets.toy_example()
print (rec.get_num_channels())


In [20]:
# output folder
output_folder = '/home/cat/code/alessio/10chan/tests/'
sorter._setup_recording(rec, 
                       output_folder)


self.params;  {'num_workers': 4}


KeyError: 'adjacency_radius'

In [5]:
# alternative to get sorter extractor and run class
sorter = YassSorter(recording=recording, output_folder=output_folder, grouping_property=grouping_property,
                         verbose=verbose, delete_output_folder=delete_output_folder)
sorter.set_params(**params)
sorter.run(raise_error=raise_error, parallel=parallel, n_jobs=n_jobs, joblib_backend=joblib_backend)
sortingextractor = sorter.get_result()

#return sortingextractor



NameError: name 'recording' is not defined

In [8]:
output_folder = '/home/cat/code/alessio/10chan/'

sorter._run(extractor, output_folder)

RUNNING SHELL SCRIPT: /home/cat/code/alessio/10chan/yass.sh


4
